In [ ]:
import $ivy.`org.typelevel::cats-core:2.1.0`

// These are all the imports you need for everything here
import cats.implicits._
import cats.data.State

# The State Monad
`cats.data.State` allows us to pass additional state around as part of a computation. It allows us to model mutable state in a purely functional way, without using mutation.

`State[S, A]` represents a function of type `S => (S, A)` where `S` is the type of the state and `A` is the type of the result.

An instance of `State` is a function that does two things:
- transforms an input state to an output state.
- computes a result.

We can "run" the `State` by providing an inital state. `State` uses `Eval` to maintain stack safety.

In [ ]:
val a = State[Int, String] { state =>
  (state, s"The state is $state")
}

// Get the state and the result:
val (state, result) = a.run(10).value

// Get the state, ignore the result:
val state2 = a.runS(10).value

// Get the result, ignore the state:
val result2 = a.runA(10).value

// The power of state comes from combining instances
val step1 = State[Int, String] { num =>
  val ans = num + 1
  (ans, s"Result of step1: $ans")
}

val step2 = State[Int, String] { num =>
  val ans = num * 2
  (ans, s"Result of step2: $ans")
}

val both = for {
  a <- step1
  b <- step2
} yield (a, b)

val (state3, result3) = both.run(20).value

Cats provides several convenience constructors for creating primitive states.

In [ ]:
// .get extracts the state as the result:
val getDemo = State.get[Int].run(10).value

// .set updates the state and returns unit as the result:
val setDemo = State.set[Int](30).run(10).value

// .pure ignores the state and returns a supplied result:
val pureDemo = State.pure[Int, String]("Result").run(10).value

// .inspect extracts the state via a tranformation function:
val inspectDemo = State.inspect[Int, String](_ + "!").run(10).value

// .modify updates the state using an update function:
val modifyDemo = State.modify[Int](_ + 1).run(10).value

We can assemble these building blocks using a for comprehension.

In [ ]:
import State._

val program: State[Int, (Int, Int, Int)] = for {
  a <- get[Int]
  _ <- set[Int](a + 1)
  b <- get[Int]
  _ <- modify[Int](_ + 1)
  c <- inspect[Int, Int](_ * 1000)
} yield (a, b, c)

val (state, result) = program.run(1).value

## Exercise
Lets implement a calculator for post-order integer arithmetic expressions. Post-order expressions are a mathematical notation where you write the operator *after* it's operands. So for example:

`1 + 2`

becomes:

`1 2 +`

Although they are more difficult for a human to read, they are easy to evaluate in code. We just need to traverse the symbols from left to right, carrying a *stack* of operands with us as we go:

- When we see a number, we push it onto the stack.
- When we see an operator, we pop two operands off the stack, operate on them, and push the result in their place.

This allows us to evaluate complex expressions without any parentheses. For example, we can evaluate `(1 + 2) * 3` as follows:

```
1 2 + 3 * // see 1, push onto stack
2 + 3 *   // see 2, push onto stack
+ 3 *     // see +, pop 1 and 2 off of stack,
          // push (1 + 2) = 3 in their place
3 3 *     // see 3, push onto stack
3 *       // see 3, push onto stack
*         // see *, pop 3 and 3 off of stack,
          // push (3 * 3) = 9 in their place
```

Lets write an interpreter for these expressions using `State`.

In [ ]:
type CalcState[A] = State[List[Int], A]

// Lets write evalOne that parses a single symbol into an instance of State. Lets not worry about error handling.
def evalOne(sym: String): CalcState[Int] =
  ???

// Here's a hint, implement evalOne in using these two helper functions
def operand(num: Int): CalcState[Int] =
  ???

def operator(func: (Int, Int) => Int): CalcState[Int] =
  ???

// Now implement evalAll by folding over the input
def evalAll(input: List[String]): CalcState[Int] =
  ???

// Finally split the input and evaluate it in evalInput
def evalInput(input: String): Int =
  ???

// Test it out. Should be 14.
val result = evalInput("4 2 / 3 4 + *")

In [ ]:
type CalcState[A] = State[List[Int], A]

// Lets write evalOne that parses a single symbol into an instance of State. Lets not worry about error handling.
def evalOne(sym: String): CalcState[Int] =
  sym match {
    case "+" => operator(_ + _)
    case "-" => operator(_ - _)
    case "*" => operator(_ * _)
    case "/" => operator(_ / _)
    case num => operand(num.toInt)
  }

// Here's a hint, implement evalOne in using these two helper functions
def operand(num: Int): CalcState[Int] =
  State[List[Int], Int] { stack =>
    (num :: stack, num)
  }

def operator(func: (Int, Int) => Int): CalcState[Int] =
  State[List[Int], Int] {
    case b :: a :: tail =>
      val ans = func(a, b)
      (ans :: tail, ans)
    case _ =>
      sys.error("Fail!")
  }

// Now implement evalAll by folding over the input
def evalAll(input: List[String]): CalcState[Int] =
  input.foldLeft(0.pure[CalcState]) { (a, b) =>
    a.flatMap(_ => evalOne(b))
  }

// Finally split the input and evaluate it in evalInput
def evalInput(input: String): Int =
  evalAll(input.split(" ").toList).runA(Nil).value

// Test it out. Should be 14.
val result = evalInput("4 2 / 3 4 + *")